In [34]:
from model.model import VanillaVAE
import numpy as np
from torch.nn import functional as F
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
#Fixes PosixPath Error
import pathlib
from data_loader.data_loaders import CelebDataLoader

temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

In [35]:
def latent_traversal(model, samples, n_changes=10, val_range=(-1, 1)):
    """ This function perform latent traversal on a VAE latent space
    model_path: str
        The absolute path of the model to load
    fname: str
        The filename to use for saving the latent traversal
    samples:
        The list of data examples to provide as input of the model
    n_changes: int
        The number of changes to perform on one latent dimension
    val_range: tuple
        The range of values that can be set for one latent dimension
    """
    # TODO: change the next two lines to retrieve the output of your encoder with pytorch
    # m = tf.keras.models.load_model(model_path)
    z_base = model.encode(samples)[-1]
    # z_base = z_base.cpu()
    # END TODO
    r, c = n_changes, z_base.shape[1]
    vals = np.linspace(*val_range, r)
    shape = samples[0].shape
    for j, z in enumerate(z_base):
        imgs = np.empty([r * c, *shape])
        for i in range(c):
            z_iter = np.tile(z, [r, 1])
            z_iter[:, i] = vals
            z_iter = torch.from_numpy(z_iter)
            # z_iter = z_iter.to(device)
            # imgs[r * i:(r * i) + r] = F.sigmoid(model.decode(z_iter)[-1])
            imgs[r * i:(r * i) + r] = (model.decode(z_iter)[-1])
        plot_traversal(imgs, r, c, shape[-1] == 1, show=True)
        # save_figure(fname, tight=False)


def plot_traversal(imgs, r, c, greyscale, show=False):
    fig = plt.figure(figsize=(160., 160.))
    grid = ImageGrid(fig, 111, nrows_ncols=(r, c), axes_pad=0, direction="column")

    for i, (ax, im) in enumerate(zip(grid, imgs)):
        ax.set_axis_off()
        im = np.transpose(im, (1,2,0))
        if i % r == 0:
            ax.set_title("z{}".format(i // r), fontdict={'fontsize': 25})
        if greyscale is True:
            ax.imshow(im, cmap="gray")
        else:
            ax.imshow(im)

    fig.subplots_adjust(wspace=0, hspace=0)
    if show is True:
        plt.show()

    plt.savefig('traversal.png')

In [36]:
data_loader = CelebDataLoader("data/",
        batch_size=1,
        shuffle=True,
        validation_split=0.0,
        # training=False,
        num_workers=2 )

In [37]:
model = VanillaVAE(3, 16)
PATH = "saved\models\VAE_CelebA\Vae_16dims.pth"
checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
state_dict = checkpoint['state_dict']
model.load_state_dict(state_dict)
model.eval()

VanillaVAE(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (4): Se

In [38]:
with torch.no_grad():
    test_input, _ = next(iter(data_loader))
    latent_traversal(model, test_input)

<Figure size 432x288 with 0 Axes>